In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Setting to use a GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
# Setting to Use a High-RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 185.7 gigabytes of available RAM

You are using a high-RAM runtime!


# Calculate PE

In [4]:
#@title PE1: nonworking(24567) PE(Seoul) (i != j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = [
    'purpose_2_with_OD_indicators.csv',
    'purpose_4_with_OD_indicators.csv',
    'purpose_5_with_OD_indicators.csv',
    'purpose_6_with_OD_indicators.csv',
    'purpose_7_with_OD_indicators.csv'
]

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for O_Seoul==1, D_Seoul==1, and O_ADMDONG_CD != D_ADMDONG_CD
df_seoul = df[
    (df['O_Seoul'] == 1) &
    (df['D_Seoul'] == 1) &
    (df['O_ADMDONG_CD'] != df['D_ADMDONG_CD'])
]

# Total count for denominator
total_cnt = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe_df = numerator.reset_index().rename(columns={'CNT': 'Numerator_CNT'})
pe_df['PE_Seoul'] = (pe_df['Numerator_CNT'] / total_cnt) * 100

# rename the PE column (Because we have different versions of PE)
pe_df = pe_df.rename(columns={'PE_Seoul': 'PE1_Seoul'})
# Save result (optional)
pe_df.to_csv(os.path.join(folder, 'PE1_within_Seoul_to_Seoul_24567_btw_notsameOD.csv'), index=False)

In [7]:
pe_df.head()

,D_ADMDONG_CD,Numerator_CNT,PE1_Seoul
0,11110515,491795.64,0.287584
1,11110530,956442.11,0.559293
2,11110540,289237.61,0.169136
3,11110550,259564.64,0.151784
4,11110560,242881.49,0.142028


In [8]:
# check the sum of PE1_Seoul
print(pe_df['PE1_Seoul'].sum())

100.0000000000002


In [5]:
#@title PE2: nonworking(24567) PE(Korea) (i != j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = [
    'purpose_2_with_OD_indicators.csv',
    'purpose_4_with_OD_indicators.csv',
    'purpose_5_with_OD_indicators.csv',
    'purpose_6_with_OD_indicators.csv',
    'purpose_7_with_OD_indicators.csv'
]

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for D_Seoul==1, and O_ADMDONG_CD != D_ADMDONG_CD
df_all_to_seoul = df[
    (df['D_Seoul'] == 1) &
    (df['O_ADMDONG_CD'] != df['D_ADMDONG_CD'])
]

# Total count for denominator
total_cnt_PE2 = df_all_to_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator_PE2 = df_all_to_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe2_df = numerator_PE2.reset_index().rename(columns={'CNT': 'Numerator_PE2_CNT'})
pe2_df['PE_all_to_seoul'] = (pe2_df['Numerator_PE2_CNT'] / total_cnt_PE2) * 100

# rename the PE column (Because we have different versions of PE)
pe2_df = pe2_df.rename(columns={'PE_all_to_seoul': 'PE2_all_to_seoul'})
# Save result (optional)
pe2_df.to_csv(os.path.join(folder, 'PE2_all_to_seoul_24567_btw_notsameOD.csv'), index=False)

In [6]:
pe2_df.head()

,D_ADMDONG_CD,Numerator_PE2_CNT,PE2_all_to_seoul
0,11110515,633332.29,0.296431
1,11110530,1166778.97,0.546111
2,11110540,348188.63,0.162970
3,11110550,317238.05,0.148483
4,11110560,299902.76,0.140370


In [8]:
# check the sum of PE2_Seoul
print(pe2_df['PE2_all_to_seoul'].sum())

99.99999999999984


In [9]:
#@title PE3: nonworking(24567) PE(Seoul) (including i=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = [
    'purpose_2_with_OD_indicators.csv',
    'purpose_4_with_OD_indicators.csv',
    'purpose_5_with_OD_indicators.csv',
    'purpose_6_with_OD_indicators.csv',
    'purpose_7_with_OD_indicators.csv'
]

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for O_Seoul==1, D_Seoul==1
df_seoul = df[
    (df['O_Seoul'] == 1) & (df['D_Seoul'] == 1)
]

# Total count for denominator
total_cnt_PE3 = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator_PE3 = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe3_df = numerator_PE3.reset_index().rename(columns={'CNT': 'Numerator_PE3_CNT'})
pe3_df['PE_Seoul'] = (pe3_df['Numerator_PE3_CNT'] / total_cnt_PE3) * 100

# rename the PE column (Because we have different versions of PE)
pe3_df = pe3_df.rename(columns={'PE_Seoul': 'PE3_Seoul'})
# Save result (optional)
pe3_df.to_csv(os.path.join(folder, 'PE3_within_Seoul_to_Seoul_24567_including_all_OD.csv'), index=False)

In [11]:
# check the sum of PE2_Seoul
print(pe3_df['PE3_Seoul'].sum())

99.99999999999983


In [12]:
#@title PE4: nonworking(24567) PE(Korea) (including i=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = [
    'purpose_2_with_OD_indicators.csv',
    'purpose_4_with_OD_indicators.csv',
    'purpose_5_with_OD_indicators.csv',
    'purpose_6_with_OD_indicators.csv',
    'purpose_7_with_OD_indicators.csv'
]

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for D_Seoul==1
df_seoul = df[(df['D_Seoul'] == 1)]

# Total count for denominator
total_cnt_PE4 = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator_PE4 = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe4_df = numerator_PE4.reset_index().rename(columns={'CNT': 'Numerator_PE4_CNT'})
pe4_df['PE_Seoul'] = (pe4_df['Numerator_PE4_CNT'] / total_cnt_PE4) * 100

# rename the PE column (Because we have different versions of PE)
pe4_df = pe4_df.rename(columns={'PE_Seoul': 'PE4_Seoul'})
# Save result (optional)
pe4_df.to_csv(os.path.join(folder, 'PE4_All_to_Seoul_24567_including_all_OD.csv'), index=False)

In [13]:
# check the sum of PE4_Seoul
print(pe4_df['PE4_Seoul'].sum())

99.99999999999994


In [14]:
#@title PE5: working trip(1) PE(Seoul) (i!=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = ['purpose_1_with_OD_indicators.csv']

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for O_Seoul==1, D_Seoul==1, and O_ADMDONG_CD != D_ADMDONG_CD
df_seoul = df[
    (df['O_Seoul'] == 1) &
    (df['D_Seoul'] == 1) &
    (df['O_ADMDONG_CD'] != df['D_ADMDONG_CD'])
]

# Total count for denominator
total_cnt = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe_df = numerator.reset_index().rename(columns={'CNT': 'Numerator_CNT'})
pe_df['PE_Seoul'] = (pe_df['Numerator_CNT'] / total_cnt) * 100

# rename the PE column (Because we have different versions of PE)
pe_df = pe_df.rename(columns={'PE_Seoul': 'PE5_Seoul'})
# Save result (optional)
pe_df.to_csv(os.path.join(folder, 'PE5_within_Seoul_to_Seoul_1working_btw_notsameOD.csv'), index=False)

In [15]:
# check the sum of PE5_Seoul
print(pe_df['PE5_Seoul'].sum())

99.99999999999996


In [16]:
#@title PE6: working trip(1) PE(Korea) (i!=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = ['purpose_1_with_OD_indicators.csv']

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for D_Seoul==1, and O_ADMDONG_CD != D_ADMDONG_CD
df_seoul = df[
    (df['D_Seoul'] == 1) &
    (df['O_ADMDONG_CD'] != df['D_ADMDONG_CD'])
]

# Total count for denominator
total_cnt = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe_df = numerator.reset_index().rename(columns={'CNT': 'Numerator_CNT'})
pe_df['PE_Seoul'] = (pe_df['Numerator_CNT'] / total_cnt) * 100

# rename the PE column (Because we have different versions of PE)
pe_df = pe_df.rename(columns={'PE_Seoul': 'PE6_Seoul'})
# Save result (optional)
pe_df.to_csv(os.path.join(folder, 'PE6_all_to_Seoul_1working_btw_notsameOD.csv'), index=False)

In [17]:
# check the sum of PE6_Seoul
print(pe_df['PE6_Seoul'].sum())

99.99999999999996


In [18]:
#@title PE7: working trip(1) PE(Seoul to Seoul) (including i=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = ['purpose_1_with_OD_indicators.csv']

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for O_Seoul == 1 & D_Seoul==1
df_seoul = df[
    (df['O_Seoul'] == 1) &
    (df['D_Seoul'] == 1)
]

# Total count for denominator
total_cnt = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe_df = numerator.reset_index().rename(columns={'CNT': 'Numerator_CNT'})
pe_df['PE_Seoul'] = (pe_df['Numerator_CNT'] / total_cnt) * 100

# rename the PE column (Because we have different versions of PE)
pe_df = pe_df.rename(columns={'PE_Seoul': 'PE7_Seoul'})
# Save result (optional)
pe_df.to_csv(os.path.join(folder, 'PE7_Seoul_to_Seoul_1working_btw_allOD.csv'), index=False)

In [19]:
# check the sum of PE7_Seoul
print(pe_df['PE7_Seoul'].sum())

100.0


In [20]:
#@title PE8: working trip(1) PE(Korea to Seoul) (including i=j)
import pandas as pd
import os

folder = '/content/drive/MyDrive/FMC_Data/M-B_data_by_purpose_domestic_residents'
files = ['purpose_1_with_OD_indicators.csv']

# Combine files
dfs = [pd.read_csv(os.path.join(folder, f)) for f in files]
df = pd.concat(dfs, ignore_index=True)

# Filter for D_Seoul==1
df_seoul = df[
    (df['D_Seoul'] == 1)
]

# Total count for denominator
total_cnt = df_seoul['CNT'].sum()

# Numerator: sum of CNT by D_ADMDONG_CD
numerator = df_seoul.groupby('D_ADMDONG_CD')['CNT'].sum()

# Calculate Pull Effect as percentage
pe_df = numerator.reset_index().rename(columns={'CNT': 'Numerator_CNT'})
pe_df['PE_Seoul'] = (pe_df['Numerator_CNT'] / total_cnt) * 100

# rename the PE column (Because we have different versions of PE)
pe_df = pe_df.rename(columns={'PE_Seoul': 'PE8_Seoul'})
# Save result (optional)
pe_df.to_csv(os.path.join(folder, 'PE8_Seoul_to_Seoul_1working_btw_allOD.csv'), index=False)


In [21]:
# check the sum of PE7_Seoul
print(pe_df['PE8_Seoul'].sum())

100.00000000000007
